# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import re
from pprint import pprint
from citipy import citipy
from datetime import datetime, timedelta

# Set today as current date and time
t = datetime.now()

# Import API key
from api_keys import g_key

# Print todays date formatted as mm/dd/yy
date = t.strftime('%m/%d/%y')
datef = t.strftime('%m%d%y')

# format time to round to the nearest hour in hundreds
time = (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
                        + timedelta(hours=t.minute//30))
hour = time.strftime('%H'+'00')

# Set a specific date and time as active parameter for CSV import
#date = '09/12/20'
#datef = '091220'

print(f'Today is {date} @ {hour} hour.')

Today is 09/17/20 @ 1200 hour.


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read CSV containing cities weather data
file = os.path.join('data', f'wx_cities_{datef}.csv')
df_wx = pd.read_csv(file)
df_wx = df_wx.drop(columns=['Unnamed: 0'])
print(f"There are {len(df_wx)} records retrieved from csvfile.")
df_wx

There are 695 records retrieved from csvfile.


,City ID,City,Country,Lat,Lon,Date (UTC),Clouds (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,3948642,Abancay,PE,-13.63,-72.88,1600360690,62,37,70.18,17.470472
1,584365,Abrau-Dyurso,RU,44.69,37.59,1600360690,0,52,75.00,9.999105
2,172515,Abu Samrah,SY,35.30,37.18,1600360690,0,38,78.87,23.107552
3,3408368,Acaraú,BR,-2.89,-40.12,1600360690,0,52,87.08,50.331067
4,2194098,Ahipara,NZ,-35.17,173.17,1600360690,98,78,57.43,27.268253
...,...,...,...,...,...,...,...,...,...,...
690,1528998,Yumen,CN,40.28,97.20,1600360824,0,30,57.38,21.273264
691,464790,Zapolyarnyy,RU,69.42,30.81,1600360824,62,65,44.60,25.523443
692,2012530,Zhigansk,RU,66.77,123.37,1600360825,0,79,39.33,4.294918
693,3191648,Zlobin,HR,45.29,14.65,1600360825,20,47,80.60,46.528275


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Copy the dataframe intro a new dataframe
df_wxh = df_wx

# drop the dataframe rows containing a null value in humidity
df_wxh.dropna(subset=['Humidity (%)'], inplace=True)
df_wxh = df_wxh.reset_index(drop=True)
print(f"There are {len(df_wxh)} cities with valid humidity conditions.")

# create variables to store the data we plan to use on our plot
# data to be used for the plotting coordinates
locations = df_wxh[["Lat","Lon"]]
# data to be used to weight the heatmap
humidity = df_wxh["Humidity (%)"]

# create the mapping figure
fig = gmaps.figure(zoom_level=1.5, center=(10,40))

# create the heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# add the heatmap layer to the figure
fig.add_layer(heat_layer)

# display the figure
fig

There are 695 cities with valid humidity conditions.


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Copy the dataframe intro a new dataframe
df_wxc = df_wx

# drop the dataframe rows containing a null value in the other parameters 
df_wxc.dropna(subset=['Humidity (%)', 'Clouds (%)', 'Max Temp (F)', 'Wind Speed (mph)'], inplace=True)
df_wxc = df_wxc.reset_index(drop=True)
print(f"There are {len(df_wxc)} cities with valid weather conditions.")
df_wxc

There are 695 cities with valid weather conditions.


,City ID,City,Country,Lat,Lon,Date (UTC),Clouds (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,3948642,Abancay,PE,-13.63,-72.88,1600360690,62,37,70.18,17.470472
1,584365,Abrau-Dyurso,RU,44.69,37.59,1600360690,0,52,75.00,9.999105
2,172515,Abu Samrah,SY,35.30,37.18,1600360690,0,38,78.87,23.107552
3,3408368,Acaraú,BR,-2.89,-40.12,1600360690,0,52,87.08,50.331067
4,2194098,Ahipara,NZ,-35.17,173.17,1600360690,98,78,57.43,27.268253
...,...,...,...,...,...,...,...,...,...,...
690,1528998,Yumen,CN,40.28,97.20,1600360824,0,30,57.38,21.273264
691,464790,Zapolyarnyy,RU,69.42,30.81,1600360824,62,65,44.60,25.523443
692,2012530,Zhigansk,RU,66.77,123.37,1600360825,0,79,39.33,4.294918
693,3191648,Zlobin,HR,45.29,14.65,1600360825,20,47,80.60,46.528275


In [5]:
# Narrow down the DataFrame to find your ideal weather condition based on the parameters set below
temp_range = [70, 80]
winds = 10
clouds = 0
ideal = []

# string variable used for file naming convention
str_cond = f'{temp_range[0]}{temp_range[1]}_{winds}_{clouds}' 

for i in range(len(df_wxc)): 
    if (df_wxc['Max Temp (F)'][i] > temp_range[0] and
        df_wxc['Max Temp (F)'][i] < temp_range[1] and
        df_wxc['Wind Speed (mph)'][i] < winds and
        df_wxc['Clouds (%)'][i] <= clouds):
                ideal.append(i)
            
# Create a new dataframe fitting weather criteria as ideal            
df_ideal = df_wxc.iloc[ideal,:]
print(f"There are {len(df_ideal)} cities with ideal weather conditions.")
df_ideal

There are 4 cities with ideal weather conditions.


,City ID,City,Country,Lat,Lon,Date (UTC),Clouds (%),Humidity (%),Max Temp (F),Wind Speed (mph)
1,584365,Abrau-Dyurso,RU,44.69,37.59,1600360690,0,52,75.00,9.999105
7,1529660,Aksu,CN,41.12,80.26,1600360691,0,23,70.77,6.800286
516,3429594,Reconquista,AR,-29.15,-59.65,1600360619,0,41,74.41,7.404259
686,1786676,Yima,CN,34.74,111.88,1600360823,0,99,71.01,4.093593


In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# create the mapping figure adding default zoom level
fig = gmaps.figure(zoom_level=1.5, center=(10,40))

# data to be used for the plotting coordinates
locations = df_ideal[["Lat","Lon"]]
    
# data to be used to weight the heatmap
temperature = df_ideal["Max Temp (F)"]

# Plot the ideal cities if any availabe from above section
if len(df_ideal) > 0:
    # create the heatmap layer
    heat_layer = gmaps.heatmap_layer(locations, weights=temperature)

    # add the heatmap layer to the figure
    fig.add_layer(heat_layer)
    
else:
    print(f"There are {len(df_ideal)} ideal cities to plot any weather heatmap.")
    
# display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `df_hotel`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Function defined to get place detail from maps api endpoint passing parameters place id and key
# Returns place list consisting of city and country name
def getplace(pid, key):
    
    # Initialize place list
    places = []
    
    # Use the search term: lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"
       
    # update address key value based on place id
    params = {
        "key": key,
        "place_id": pid
    }
        
    # make request
    g_places = requests.get(base_url, params=params)
    
    # convert to json
    g_places = g_places.json()
        
    try:   
        # set comps to the address components within the json places api
        comps = g_places["result"]["address_components"]
        for i in comps:
            # if the address component type is a 'locality' then assign its long name format as city name
            if i["types"][0] == 'locality':
                cityname = i["long_name"]
            # if the address component type is a 'country' then assign its long name format as country name
            if i["types"][0] == 'country':
                countryname = i["long_name"]
                
        # add both fields retrived to the places list
        places.append(cityname)
        places.append(countryname)
        
    except (KeyError, IndexError, ValueError):
        print("Missing field/result... skipping.")
    
    # return places from this function when called
    return places   

In [8]:
# Create two new list with hotel names and city id (from ideal dataframe)
g_hotel = []
g_city = []
g_country = []
g_lat = []
g_lon = []

# Add an empty hotel column to heatmap dataframe
df_wxh['Hotel Name'] = ""

# create a params dict that will be updated with new city each iteration
params = {"key": g_key,
          "radius": 5000,
          "type": "lodging"}


print("Beginning Hotel Data Retrieval")
print('-'*40)

# Loop through the ideal weather cities
# Run a lat/long search for each city with the closest hotel listing within the radius in meters
for index, row in df_ideal.iterrows():
    
    lat = row['Lat']
    lon = row['Lon']
    print(f"Searching within {row['City']}, {row['Country']} using coordinates ({lat},{lon})...")
    
    # update address key value based on coordinates
    params['location'] = f"{lat},{lon}"
    
    # Use the search term: lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    # make request and print url
    g_cities = requests.get(base_url, params=params)

    # convert to json
    g_cities = g_cities.json()

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:    
        name = g_cities["results"][0]["name"]
        
        place = g_cities["results"][0]["place_id"]
        
        #c = g_cities["results"][0]["plus_code"]["compound_code"]
        #cn = re.split(',',c)
        #cn = cn[len(cn)-1]
        #country = cn[cn.find(' ')+1:]
             
        #c = g_cities["results"][0]["vicinity"]
        #cn = re.split(',',c)
        #cn = cn[len(cn)-1]
        #city = cn[cn.find(' ')+1:]
                
        lat = g_cities["results"][0]["geometry"]["location"]["lat"]
        lon = g_cities["results"][0]["geometry"]["location"]["lng"]
        
        place = getplace(place,g_key)
        
        g_hotel.append(name)
        g_city.append(place[0])
        g_country.append(place[1])
        g_lat.append(lat)
        g_lon.append(lon)
        
    except (KeyError, IndexError, ValueError):
        print("Missing field/result... skipping.")
        continue
 
        
hotel_data = {
    "Hotel Name": g_hotel,
    "Lat": g_lat,
    "Lon": g_lon,
    "City": g_city,
    "Country": g_country
}
    
df_hotels = pd.DataFrame(hotel_data)    
        
print('-'*40)
print("Data Retrieval Complete")
print('-'*40)   

Beginning Hotel Data Retrieval
----------------------------------------
Searching within Abrau-Dyurso, RU using coordinates (44.69,37.59)...
Searching within Aksu, CN using coordinates (41.12,80.26)...
Searching within Reconquista, AR using coordinates (-29.15,-59.65)...
Searching within Yima, CN using coordinates (34.74,111.88)...
----------------------------------------
Data Retrieval Complete
----------------------------------------


In [9]:
# Print number of hotels and its detail
print(f"There are {len(df_hotels)} hotel(s) within a radius of {params['radius']} meters of ideal weather conditions.")

# Set the file path to export hotel data
csvFile = os.path.join("data/hotels",f"wx_hotels_{datef}_{hour}_{str_cond}.csv")

# Export datframe to csv file
df_hotels.to_csv(csvFile)

df_hotels

There are 4 hotel(s) within a radius of 5000 meters of ideal weather conditions.


,Hotel Name,Lat,Lon,City,Country
0,Avgustin Inn,44.714546,37.592597,Abrau-Dyurso,Russia
1,Pudong Holiday Hotel,41.141760,80.291396,Akesu Diqu,China
2,Grand Hotel,-29.146211,-59.645088,Reconquista,Argentina
3,Yima Hotel,34.747646,111.872380,Sanmenxia Shi,China


In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# create variables to store the data we plan to use on our plot
# data to be used for the plotting coordinates
locations = df_wxh[["Lat","Lon"]]
# data to be used to weight the heatmap
humidity = df_wxh["Humidity (%)"]

# create the mapping figure adding default zoom level
fig = gmaps.figure(zoom_level=1.8, center=(10,40))

# create the heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# add the heatmap layer to the figure
fig.add_layer(heat_layer)

# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotels.iterrows()]
locations = df_hotels[["Lat", "Lon"]]

# create the markers using hotel list to fill the info box
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# add the markers to the figure
fig.add_layer(marker)

# display the figure
fig

Figure(layout=FigureLayout(height='420px'))